In [1]:
!pip install chromadb google-generativeai pandas python-dotenv

In [2]:
import json
import pandas as pd
import chromadb
import google.generativeai as genai
import time
import os
from dotenv import load_dotenv


/Users/nachiket/workspace/github.com/Public/tux-to-go-poc/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

genai.configure(api_key=os.getenv("GEMINI_API_KEY")) # Or set GOOGLE_API_KEY environment variable


In [4]:
def load_and_prepare_docs(filepath="codebase_map.jsonl"):
    """Loads the JSONL file and formats each entry for embedding."""
    documents = []
    metadata = []
    ids = []
    with open(filepath, 'r') as f:
        for i, line in enumerate(f):
            data = json.loads(line)
            content = ""

            # This part is the fix: creating a simple dictionary
            simple_meta = {
                "file_path": data.get("file_path", ""),
                "type": data.get("type", "")
            }

            if data['type'] == 'function':
                func = data.get('function', {})
                content = f"File: {data['file_path']}\nType: function\nSignature: {func.get('signature', '')}\nBody: {func.get('body', '')}"
                simple_meta["name"] = func.get("name", "")
            elif data['type'] == 'struct':
                struct = data.get('struct', {})
                fields_str = "\n".join([f"  {field.get('name', '')} {field.get('type', '')} `{field.get('tag', '')}`" for field in struct.get('fields', [])])
                content = f"File: {data['file_path']}\nType: struct\nName: {struct.get('name', '')}\nFields:\n{fields_str}"
                simple_meta["name"] = struct.get("name", "")

            documents.append(content)
            metadata.append(simple_meta) # We append the simple metadata here
            ids.append(str(i + 1))

    return documents, metadata, ids

print("Loading and preparing documents...")
documents, metadata, ids = load_and_prepare_docs()
print(f"Loaded {len(documents)} documents.")
# Example of a prepared document:
print(documents[5])

Loading and preparing documents...
Loaded 99 documents.
File: ../codebase/runMain.go
Type: function
Signature: func parseFile(path string, encoder *json.Encoder)
Body: 
	fset := token.NewFileSet()
	fileBytes, err := os.ReadFile(path)
	if err != nil {
		fmt.Printf("Warning: could not read file %s: %v\n", path, err)
		return
	}

	node, err := parser.ParseFile(fset, path, fileBytes, parser.ParseComments)
	if err != nil {
		fmt.Printf("Warning: could not parse file %s: %v\n", path, err)
		return
	}

	ast.Inspect(node, func(n ast.Node) bool {
		// 2. MODIFICATION: Use a type switch to handle different declaration types.
		switch decl := n.(type) {
		case *ast.FuncDecl:
			// Handle function declarations
			if decl.Body == nil {
				return true // Skip function declarations without a body (e.g., in interfaces)
			}

			funcMeta := FunctionMetadata{
				Name:       decl.Name.Name,
				Parameters: extractFields(decl.Type.Params, fileBytes),
				Returns:    extractFields(decl.Type.Results, file

Embed and Store in VectorDB

In [5]:
# Initialize ChromaDB client.
client = chromadb.PersistentClient(path="./chroma_db")
collection_name = "gocodebase"

# 1. Delete the collection if it already exists to ensure a fresh start
if collection_name in [c.name for c in client.list_collections()]:
    client.delete_collection(name=collection_name)
    print(f"Deleted existing collection: '{collection_name}'")

# 2. Create a new, empty collection
collection = client.create_collection(name=collection_name)
print(f"Created a new collection: '{collection_name}'")

# 3. Index the documents (this code now runs every time)
print("Embedding and indexing the codebase... This may take a moment.")
batch_size = 50 # Increased batch size for efficiency
for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]
    batch_meta = metadata[i:i+batch_size]

    response = genai.embed_content(
        model="models/text-embedding-004",
        content=batch_docs,
        task_type="RETRIEVAL_DOCUMENT"
    )
    embeddings = response['embedding']

    collection.add(
        embeddings=embeddings,
        documents=batch_docs,
        metadatas=batch_meta,
        ids=batch_ids
    )
    print(f"Indexed batch {i//batch_size + 1}...")
    time.sleep(1) # Reduced sleep time

print("Codebase successfully indexed in ChromaDB.")
item_count = collection.count()
print(f"The collection now has {item_count} items.")

Deleted existing collection: 'gocodebase'
Created a new collection: 'gocodebase'
Embedding and indexing the codebase... This may take a moment.


E0000 00:00:1759672845.326794 4447472 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Indexed batch 1...
Indexed batch 2...
Codebase successfully indexed in ChromaDB.
The collection now has 99 items.


In [6]:
# Add this cell to verify the contents of your ChromaDB collection
client = chromadb.PersistentClient(path="./chroma_db")
try:
    collection = client.get_collection(name="gocodebase")
    item_count = collection.count()
    print(f"The collection '{collection.name}' has {item_count} items.")

    if item_count > 0:
        print("\nHere's a sample of the data in the collection:")
        # Peek at the first 2 items to ensure they look correct
        sample = collection.peek(limit=2)
        print(sample['documents'])
except ValueError:
    print("The collection 'gocodebase' does not exist. Please run the indexing cell first.")

The collection 'gocodebase' has 99 items.

Here's a sample of the data in the collection:
['File: ../codebase/runMain.go\nType: struct\nName: Field\nFields:\n  Name string `json:"name"`\n  Type string `json:"type"`\n  Tag string `json:"tag,omitempty"`', 'File: ../codebase/runMain.go\nType: struct\nName: FunctionMetadata\nFields:\n  Name string `json:"name"`\n  Signature string `json:"signature"`\n  Parameters []Field `json:"parameters"`\n  Returns []Field `json:"returns"`\n  Body string `json:"body"`']


Take request, find relevent snippets from chroma nad pass to generative model

In [7]:
def query_rag(query: str, n_results: int = 5):
    """Performs the RAG process: query -> retrieve -> augment -> generate."""

    # 1. Retrieve relevant code snippets
    # Embed the user's query
    query_embedding_response = genai.embed_content(
        model="models/text-embedding-004",
        content=query,
        task_type="RETRIEVAL_QUERY"
    )

    # Search ChromaDB for the most similar documents
    results = collection.query(
        query_embeddings=[query_embedding_response['embedding']],
        n_results=n_results
    )

    # Print the results from ChromaDB for debugging
    print("ChromaDB query results:")
    print(results)
    print("---")

    retrieved_docs = results['documents'][0] if results and results['documents'] and results['documents'][0] else []
    context = "\n---\n".join(retrieved_docs)


    # 2. Augment: Create a prompt for the generative model
    prompt = f"""You are an expert Go programmer. Your task is to help a user modify their codebase.
Use the following relevant code snippets from the codebase as context to provide a complete and accurate answer.Acknowledge the lack of code snippets if there is none.

**CONTEXT FROM THE CODEBASE:**
---
{context}
---

**USER'S REQUEST:**
"{query}"

**YOUR TASK:**
Based on the user's request and the provided context, generate the necessary code changes.
- If a struct needs modification, show the new struct definition.
- If a function needs to be changed, provide the complete, updated function body.
- If new functions are needed, write them.
- Provide a brief, clear explanation of the changes you made.
- Present the final output in Go code blocks.
"""
    if context == "":
        return "no context"
    # 3. Generate the response
    model = genai.GenerativeModel('gemini-2.0-flash')
    response = model.generate_content(prompt)

    return response.text

Giving request


In [8]:
user_request = """
I need to add a 'likes' count to the Chirp model.
It should be an integer and default to 0.

Then, update the 'handlerChirpsCreate' function. After creating a chirp,
the response should include this new 'likes' field.
"""

# Get the suggested code change
suggested_change = query_rag(user_request)

# Print the result
print("--- SUGGESTED CODE CHANGE ---")
print(suggested_change)

ChromaDB query results:
{'ids': [['17', '10', '16', '48', '86']], 'embeddings': None, 'documents': [['File: ../handler_chirps_get.go\nType: function\nSignature: func handlerChirpsRetrieve(w http.ResponseWriter, r *http.Request)\nBody: \n\tdbChirps, err := cfg.db.GetChirps(r.Context())\n\tif err != nil {\n\t\trespondWithError(w, http.StatusInternalServerError, "Couldn\'t retrieve chirps", err)\n\t\treturn\n\t}\n\n\tchirps := []Chirp{}\n\tfor _, dbChirp := range dbChirps {\n\t\tchirps = append(chirps, Chirp{\n\t\t\tID:        dbChirp.ID,\n\t\t\tCreatedAt: dbChirp.CreatedAt,\n\t\t\tUpdatedAt: dbChirp.UpdatedAt,\n\t\t\tUserID:    dbChirp.UserID,\n\t\t\tBody:      dbChirp.Body,\n\t\t})\n\t}\n\n\trespondWithJSON(w, http.StatusOK, chirps)\n}\n', 'File: ../handler_chirps_create.go\nType: struct\nName: Chirp\nFields:\n  ID uuid.UUID `json:"id"`\n  CreatedAt time.Time `json:"created_at"`\n  UpdatedAt time.Time `json:"updated_at"`\n  UserID uuid.UUID `json:"user_id"`\n  Body string `json:"body"`'

In [9]:
import google.generativeai as genai

# Make sure your key is configured via one of the methods above
# genai.configure(api_key="...")

# This loop will print available models if your key is valid
# It will fail with the same error if the key is still invalid
print("Verifying API Key by listing available models:")
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

Verifying API Key by listing available models:


E0000 00:00:1759672856.405982 4447472 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001
